<h2>Лабораторная работа №6 по курсу "Методы машинного обучения"</h2>

<p><b>Выполнил:</b> Арбузов А.П. группа ИУ5-24М</p>

<h3>Задание</h3>
<p>Для произвольного набора данных, предназначенного для классификации текстов, решите задачу классификации текста двумя способами:</p>
<ol>
    <li>На основе CountVectorizer или TfidfVectorizer.</li>
    <li>На основе моделей word2vec или Glove или fastText.</li>
</ol> 
<p>Сравните качество полученных моделей.</p>

In [11]:
import re
import pandas as pd
import numpy as np
from typing import Dict, Tuple
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from nltk import WordPunctTokenizer
from nltk.corpus import stopwords
import nltk
import gensim
from gensim.models import word2vec
nltk.download('stopwords')
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package stopwords to C:\Users\Alexei
[nltk_data]     Arbuzov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
data = pd.read_csv('Spam.csv', sep = ',')
labelEnc = LabelEncoder()
data[['Category']] = pd.DataFrame(labelEnc.fit_transform(data['Category'].astype(str)), columns=['Category'])
data.head(10)

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
5,1,FreeMsg Hey there darling it's been 3 week's n...
6,0,Even my brother is not like to speak with me. ...
7,0,As per your request 'Melle Melle (Oru Minnamin...
8,1,WINNER!! As a valued network customer you have...
9,1,Had your mobile 11 months or more? U R entitle...


<h3>На основе CountVectorizer</h3>

In [13]:
content = data['Message']
target = data['Category']

TrainX, TestX, TrainY, TestY = train_test_split(content, target, test_size=0.3, random_state = 1)

In [24]:
model = Pipeline(
        [("vectorizer", CountVectorizer()), 
         ("classifier", RandomForestClassifier())])
model.fit(TrainX, TrainY)
print_accuracy_score_for_classes(TestY, model.predict(TestX))

Метка 	 Accuracy
0 	 0.9986130374479889
1 	 0.8173913043478261


<h3>На основе Word2vec</h3>

In [18]:
corpus = []
stop_words = stopwords.words('english')
tok = WordPunctTokenizer()
for line in data['Message'].values:
    line1 = line.strip().lower()
    line1 = re.sub("[^a-zA-Z]"," ", line1)
    text_tok = tok.tokenize(line1)
    text_tok1 = [w for w in text_tok if not w in stop_words]
    corpus.append(text_tok1)

In [19]:
corpus[5:10]

[['freemsg',
  'hey',
  'darling',
  'week',
  'word',
  'back',
  'like',
  'fun',
  'still',
  'tb',
  'ok',
  'xxx',
  'std',
  'chgs',
  'send',
  'rcv'],
 ['even', 'brother', 'like', 'speak', 'treat', 'like', 'aids', 'patent'],
 ['per',
  'request',
  'melle',
  'melle',
  'oru',
  'minnaminunginte',
  'nurungu',
  'vettam',
  'set',
  'callertune',
  'callers',
  'press',
  'copy',
  'friends',
  'callertune'],
 ['winner',
  'valued',
  'network',
  'customer',
  'selected',
  'receivea',
  'prize',
  'reward',
  'claim',
  'call',
  'claim',
  'code',
  'kl',
  'valid',
  'hours'],
 ['mobile',
  'months',
  'u',
  'r',
  'entitled',
  'update',
  'latest',
  'colour',
  'mobiles',
  'camera',
  'free',
  'call',
  'mobile',
  'update',
  'co',
  'free']]

In [20]:
model_imdb = word2vec.Word2Vec(corpus, workers=4, min_count=10, window=10, sample=1e-3)

In [21]:
class EmbeddingVectorizer(object):
    '''
    Для текста усредним вектора входящих в него слов
    '''
    def __init__(self, model):
        self.model = model
        self.size = model.vector_size

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([np.mean(
            [self.model[w] for w in words if w in self.model] 
            or [np.zeros(self.size)], axis=0)
            for words in X])

In [22]:
def accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray) -> Dict[int, float]:
    """
    Вычисление метрики accuracy для каждого класса
    y_true - истинные значения классов
    y_pred - предсказанные значения классов
    Возвращает словарь: ключ - метка класса, 
    значение - Accuracy для данного класса
    """
    # Для удобства фильтрации сформируем Pandas DataFrame 
    d = {'t': y_true, 'p': y_pred}
    df = pd.DataFrame(data=d)
    # Метки классов
    classes = np.unique(y_true)
    # Результирующий словарь
    res = dict()
    # Перебор меток классов
    for c in classes:
        # отфильтруем данные, которые соответствуют 
        # текущей метке класса в истинных значениях
        temp_data_flt = df[df['t']==c]
        # расчет accuracy для заданной метки класса
        temp_acc = accuracy_score(
            temp_data_flt['t'].values, 
            temp_data_flt['p'].values)
        # сохранение результата в словарь
        res[c] = temp_acc
    return res

def print_accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray):
    """
    Вывод метрики accuracy для каждого класса
    """
    accs = accuracy_score_for_classes(y_true, y_pred)
    if len(accs)>0:
        print('Метка \t Accuracy')
    for i in accs:
        print('{} \t {}'.format(i, accs[i]))

In [23]:
model2 = Pipeline(
        [("vectorizer", EmbeddingVectorizer(model_imdb.wv)), 
         ("classifier", RandomForestClassifier())])
model2.fit(TrainX,TrainY)
print_accuracy_score_for_classes(TestY, model2.predict(TestX))

Метка 	 Accuracy
0 	 0.9889042995839112
1 	 0.4608695652173913
